## Training GMM Model from PKL files
## GMM - CQCC
## GMM - MFCC

In [1]:
import os
import numpy as np
import pickle
import argparse
import sys
import time
import multiprocessing as mp
import warnings
import pandas as pd
from sklearn.mixture import GaussianMixture as GMM
from sklearn.svm import SVC

In [2]:
root_path = 'Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/'


In [3]:
train_features_path = '/Users/asimadnan/Desktop/Mres/Experiments/testfiles/la_train_mfcc_cqcc.pkl'
dev_features_path = '/Users/asimadnan/Desktop/Mres/Experiments/testfiles/la_dev_mfcc_cqcc.pkl'
output_path = '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/output_data/'

# sample small file to test evrything, comment this line after all tests run correctly
train_features_path = '/Users/asimadnan/Desktop/Mres/Experiments/testfiles/04-04-2021_15-09_dev_1.pkl'
dev_features_path = '/Users/asimadnan/Desktop/Mres/Experiments/testfiles/04-04-2021_15-09_dev_1.pkl'

### Extract CQCC from train & dev

In [7]:
def extract_feature(pkl_path,feature_type):
    max_len = 50  # 1.25 seconds  # check the timesteps of cqcc and mfcc 
    X = []
    y = []
    i=1
    with open(pkl_path, 'rb') as infile:
        data = pickle.load(infile)
        total_files = len(data)
        for feat_cqcc, feat_mfcc, label,filename in data:

            features = []
            feature_block = ''
           
            if feature_type == 'mfcc':
                feature_block = feat_mfcc
            elif feature_type == 'cqcc':
                
                feature_block = feat_cqcc


            if len(feature_block) > max_len:
                features = feature_block[:max_len]
            elif len(feature_block) < max_len:
                features = np.concatenate((feature_block, np.array([[0.]*num_dim]*(max_len-len(feature_block)))), axis=0)
            
            if (i%2 == 0):
                print( ((i/total_files)*100), ' % done' )
            i+=1
            print( ((i/total_files)*100), ' % done' )
            X.append(features.reshape(-1))
            y.append([label,filename])
        return X,y

In [8]:
# Change this acordingly, these are used to set the file name
feature = 'cqcc'
dataset = 'test'
###

X,y = extract_feature(dev_features_path,feature)
train_cqcc = np.append(np.array(X),np.array(y),1)
pd.DataFrame(train_cqcc).to_csv(os.path.join(output_path + dataset + '__' + feature + '.csv'))

28.57142857142857  % done
28.57142857142857  % done
42.857142857142854  % done
57.14285714285714  % done
57.14285714285714  % done
71.42857142857143  % done
85.71428571428571  % done
85.71428571428571  % done
100.0  % done
114.28571428571428  % done


In [9]:
# Change this acordingly, these are used to set the file name
feature = 'mfcc'
dataset = 'test'
###

X,y = extract_feature(dev_features_path,feature)
train_cqcc = np.append(np.array(X),np.array(y),1)
pd.DataFrame(train_cqcc).to_csv(os.path.join(output_path + dataset + '__' + feature + '.csv'))

28.57142857142857  % done
28.57142857142857  % done
42.857142857142854  % done
57.14285714285714  % done
57.14285714285714  % done
71.42857142857143  % done
85.71428571428571  % done
85.71428571428571  % done
100.0  % done
114.28571428571428  % done


In [28]:
############ 
#testing if created file is in correct format
abc = pd.read_csv(output_path + dataset + '_' + feature + '.csv',index_col=0)
########

In [10]:
# training GMM 

In [47]:
train_data_cqcc_path = '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/output_data/train_cqcc.csv'
train_data_mfcc_path = '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/output_data/train_mfcc.csv'
train_label_path = '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt'

dev_data_cqcc_path = '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/output_data/dev_cqcc.csv'
dev_data_mfcc_path = '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/output_data/dev_mfcc.csv'
dev_label_path = '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.dev.trl.txt'



# train_data_mfcc_path = '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/output_data/test__mfcc.csv'
# train_data_cqcc_path= '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/output_data/test__cqcc.csv'




In [48]:
train = [train_data_cqcc_path,train_data_mfcc_path,train_label_path,'train']
dev = [dev_data_cqcc_path,dev_data_mfcc_path,dev_label_path,'dev']

In [50]:
dataset = 'dev'
feature = 'cqcc'
labels = pd.read_csv(dev_label_path, delimiter = " ", header=None)
labels.columns = ['SPEAKER_ID','AUDIO_FILE_NAME','ENVIRONMENT_ID','ATTACK_ID','KEY']
labels


,SPEAKER_ID,AUDIO_FILE_NAME,ENVIRONMENT_ID,ATTACK_ID,KEY
0,LA_0069,LA_D_1047731,-,-,bonafide
1,LA_0069,LA_D_1105538,-,-,bonafide
2,LA_0069,LA_D_1125976,-,-,bonafide
3,LA_0069,LA_D_1293230,-,-,bonafide
4,LA_0069,LA_D_1340209,-,-,bonafide
...,...,...,...,...,...
24839,LA_0078,LA_D_9924204,-,A06,spoof
24840,LA_0078,LA_D_9931163,-,A06,spoof
24841,LA_0078,LA_D_9935163,-,A06,spoof
24842,LA_0078,LA_D_9944718,-,A06,spoof


In [52]:
feature_len = 3000

data = pd.read_csv(dev_data_cqcc_path,index_col=0)
data = data.rename({str(feature_len + 1): 'AUDIO_FILE_NAME'}, axis=1)
# data = pd.merge(data, labels, on='AUDIO_FILE_NAME')
data

,0,1,2,3,4,5,6,7,8,9,...,2992,2993,2994,2995,2996,2997,2998,2999,3000,AUDIO_FILE_NAME
0,-660.349393,-704.826919,-737.667327,-758.736888,-768.571412,-768.312145,-759.600974,-744.443859,-725.054528,-703.688437,...,0.001909,0.002105,0.002208,0.002210,0.002106,0.001896,0.001586,0.001187,spoof,LA_D_2776056
1,-607.224471,-648.401295,-678.918875,-698.643791,-708.056697,-708.194137,-700.552968,-686.965561,-669.456783,-650.091891,...,-0.046892,-0.048066,-0.048432,-0.047955,-0.046637,-0.044525,-0.041700,-0.038282,spoof,LA_D_8676372
2,-496.906341,-530.001525,-554.334842,-569.845949,-576.980515,-576.638737,-570.093151,-558.882739,-544.692616,-529.227138,...,0.000619,0.000747,0.000885,0.001027,0.001169,0.001303,0.001425,0.001530,spoof,LA_D_4086358
3,-738.769741,-788.531735,-825.242460,-848.749818,-859.653368,-859.232618,-849.329530,-832.195300,-810.314892,-786.220561,...,-0.017403,-0.015597,-0.013639,-0.011627,-0.009660,-0.007826,-0.006202,-0.004846,spoof,LA_D_4407621
4,-550.008090,-586.954116,-614.166072,-631.542827,-639.545115,-639.140930,-631.716649,-618.961487,-602.735463,-584.929406,...,-0.000037,-0.000165,-0.000309,-0.000469,-0.000639,-0.000816,-0.000994,-0.001165,spoof,LA_D_6154817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24839,-455.021064,-485.527792,-507.963987,-522.242437,-528.741305,-528.259467,-521.943456,-511.192271,-497.548388,-482.582000,...,0.002785,0.003084,0.003184,0.003107,0.002882,0.002549,0.002149,0.001727,spoof,LA_D_8379770
24840,-847.107768,-904.187984,-946.307529,-973.295805,-985.845225,-985.428713,-974.164608,-954.640489,-929.711429,-902.285631,...,-0.001272,-0.001592,-0.001791,-0.001863,-0.001809,-0.001640,-0.001373,-0.001030,spoof,LA_D_7522206
24841,-717.871323,-766.108676,-801.643551,-824.333999,-834.770043,-834.203605,-824.433850,-807.657728,-786.298859,-762.825739,...,-0.000848,-0.000711,-0.000506,-0.000252,0.000026,0.000299,0.000539,0.000717,spoof,LA_D_7830688
24842,-779.999560,-832.796768,-871.888866,-897.116233,-909.110096,-909.216713,-899.373360,-881.946732,-859.547974,-834.836242,...,0.010334,0.010611,0.011116,0.011781,0.012525,0.013263,0.013912,0.014398,spoof,LA_D_3449140


In [53]:
data = None

In [39]:
X = data.iloc[:, 0:feature_len+1]
X

,0,1,2,3,4,5,6,7,8,9,...,641,642,643,644,645,646,647,648,649,650
0,-8.573016,-11.678742,3.349419,6.297206,6.607597,6.585302,1.953112,3.147527,-2.786101,4.067646,...,8.802019,4.184361,10.452165,6.674898,6.890626,0.663099,-2.561426,-8.769751,-9.683469,spoof
1,-14.455932,-31.868080,-4.644265,-2.268237,7.669049,7.925644,17.380247,17.756672,15.522497,17.359138,...,19.509102,12.791993,14.121706,15.612497,16.527843,16.433516,9.285261,4.986468,4.088097,spoof
2,-12.320907,-11.391042,6.357327,2.426181,6.995060,4.154011,7.580681,12.936386,8.104214,-4.446559,...,-18.018298,-8.785583,10.563321,-23.634459,1.680121,5.230506,0.680723,-23.921085,-29.724428,spoof
3,-9.393365,-5.249382,-3.219313,2.639915,-11.203670,-16.226921,-35.164506,-24.291069,3.423613,13.295148,...,-14.981033,-32.174932,-29.298206,-24.861730,-13.446369,2.874312,0.286864,10.052436,4.298754,spoof
4,-11.876866,-4.572194,14.096091,-6.980273,-4.931390,-17.709929,-15.360505,-11.763362,-4.758243,2.240678,...,-8.474299,-19.370402,-23.833592,-15.762516,-9.099456,1.400025,-3.759530,-2.098907,-5.222501,spoof
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25375,-11.085313,-7.561798,-1.381273,1.188675,1.447115,-2.059506,-3.037639,-2.829265,-4.664765,-6.016820,...,-39.860679,-18.460338,-30.597868,-7.262566,-0.994769,-7.309093,6.028889,-11.205987,9.621982,spoof
25376,-15.204085,-28.744689,-2.998543,-3.009965,6.409241,4.878647,13.757629,12.325082,10.477858,14.083744,...,23.428317,16.784133,15.449017,19.464919,12.815936,5.722556,2.030922,2.933079,7.115255,spoof
25377,-11.153386,-9.941480,7.826534,0.958543,4.991591,13.767345,13.991147,10.950425,3.941666,-2.001952,...,-0.640232,8.237400,16.081417,15.103763,0.715768,4.415959,10.667828,-7.880399,-3.173169,bonafide
25378,-9.084023,-10.608522,5.265818,1.810717,7.086166,10.198746,11.721395,10.058237,4.518279,18.689861,...,7.700274,0.467699,0.093255,2.616595,1.005538,3.728195,4.203430,-2.405135,0.471586,spoof


In [40]:
bondata = X.loc[X[str(feature_len)] == 'bonafide'].iloc[:, 0:feature_len]
spoofdata = X.loc[X[str(feature_len)] == 'spoof'].iloc[:, 0:feature_len]

gmm_bon = GMM(n_components = 1, covariance_type='diag',n_init = 50,verbose=1) # min shape[0] = 135 # max = 1112
gmm_sp  = GMM(n_components = 1, covariance_type='diag',n_init = 50,verbose=1)  # min shape[0] = 64  # max = 1318

In [41]:
bondata

,0,1,2,3,4,5,6,7,8,9,...,640,641,642,643,644,645,646,647,648,649
6,-10.755181,-6.555706,8.419038,-3.742073,-0.287701,-0.453065,2.295795,3.699362,4.668490,5.384911,...,-2.481737,-2.757649,-5.678154,2.275743,-1.486575,-1.312862,-4.243270,-12.243520,-2.557074,-2.787862
17,-11.723814,-9.750538,10.354313,0.791462,1.879989,11.423931,13.637714,18.582008,18.859869,6.030119,...,-0.585245,8.078474,10.853899,17.137064,23.867663,16.145534,8.275949,3.760915,4.905719,11.029994
19,-12.145011,-11.711807,13.580812,6.908806,8.691896,8.261685,7.357390,10.946545,4.654772,7.016511,...,8.299290,9.725975,12.247905,15.429032,15.522858,13.873048,14.220463,8.509489,-1.277293,4.128119
54,-12.322505,-8.670858,11.619966,4.731583,7.153312,6.476455,3.802160,10.528387,0.205263,-8.374908,...,10.396177,7.475150,9.065025,5.005353,13.590110,9.017063,7.123059,4.110650,-2.214155,7.571899
67,-11.771877,-6.049785,5.078198,9.735997,12.197072,2.994727,10.592675,12.217650,8.667267,5.794768,...,0.560323,4.087174,7.923929,12.558359,14.957126,5.585334,11.335784,11.383738,8.454517,-3.668787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25359,-10.062475,-7.352747,5.347971,1.178981,2.068914,2.484618,4.784619,2.790660,4.046308,1.600950,...,8.865943,5.850477,10.412114,14.327317,11.453071,3.903163,1.596728,-0.978471,4.961662,3.431231
25367,-12.084336,-8.529256,13.751033,6.729279,13.259378,12.264877,13.000313,13.020607,18.250294,12.477184,...,6.132513,14.820159,19.181104,18.864175,11.314332,10.069025,5.086636,0.071582,-2.084654,0.404231
25370,-12.493473,-11.548445,11.359798,2.413646,1.158224,1.849956,2.446304,4.636609,6.265790,-4.825328,...,0.719816,8.600658,13.550634,10.081595,2.410478,-4.244541,-4.866430,4.341663,7.344098,5.437800
25373,-10.446565,-6.208696,1.932516,-4.192936,7.615052,2.243147,-0.621025,1.908519,10.144596,10.692419,...,-8.551895,0.232270,1.861281,3.251841,4.229142,19.689048,16.977596,-3.517560,-2.776402,3.675637


In [42]:
t0 = time.time()
print('Training Bonafine Gmm on',feature, dataset)

gmm_bon.fit(bondata)
print('Bonafide gmm trained, time spend:', time.time() - t0)
#pickle.dump(gmm_bon, open(output_path + dataset + '__' + feature + '_bon.gmm', 'wb'))


# t0 = time.time()
# print('Training Spoof Gmm on',feature, dataset)
# gmm_sp.fit(spoofdata)
# print('Spoof gmm trained, time spend:', time.time() - t0)
# pickle.dump(gmm_sp, open(output_path + dataset + '__' + feature + '_spoof.gmm', 'wb'))

Training Bonafine Gmm on mfcc train
Initialization 0
Initialization converged: True
Initialization 1
Initialization converged: True
Initialization 2
Initialization converged: True
Initialization 3
Initialization converged: True
Initialization 4
Initialization converged: True
Initialization 5
Initialization converged: True
Initialization 6
Initialization converged: True
Initialization 7
Initialization converged: True
Initialization 8
Initialization converged: True
Initialization 9
Initialization converged: True
Initialization 10
Initialization converged: True
Initialization 11
Initialization converged: True
Initialization 12
Initialization converged: True
Initialization 13
Initialization converged: True
Initialization 14
Initialization converged: True
Initialization 15
Initialization converged: True
Initialization 16
Initialization converged: True
Initialization 17
Initialization converged: True
Initialization 18
Initialization converged: True
Initialization 19
Initialization converged:

In [ ]:
#SVM


In [57]:
output_path = '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/output_data/'


train_data_cqcc_path = '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/output_data/train_cqcc.csv'
train_data_mfcc_path = '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/output_data/train_mfcc.csv'
train_label_path = '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt'


train_data_mfcc_path = '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/output_data/test__mfcc.csv'
train_data_cqcc_path= '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/output_data/test__cqcc.csv'

In [58]:
train = [train_data_cqcc_path,train_data_mfcc_path,train_label_path,'train']
features = ['cqcc','mfcc']

In [59]:
data = pd.read_csv(train[1],index_col=0)

In [60]:
data

,0,1,2,3,4,5,6,7,8,9,...,642,643,644,645,646,647,648,649,650,651
0,-12.540241,-3.133893,18.544395,-3.986809,-7.377833,-25.542547,-25.324949,-11.175202,-4.969218,-0.132238,...,-26.830448,-27.055918,-12.060779,-8.749569,-4.577082,-0.492432,-2.774876,-6.726992,spoof,LA_D_1099405
1,-10.922028,-8.007983,4.348195,-0.213975,2.816931,1.710826,2.416150,4.800140,5.433692,-0.691564,...,2.829533,6.430730,9.741274,9.693492,4.246516,1.414347,7.400756,4.200158,bonafide,LA_D_9330492
2,-12.298278,-9.924849,13.882997,7.079037,5.279849,4.095138,8.002585,11.920810,7.627260,2.259768,...,12.752793,12.872806,17.628176,11.319846,3.908065,2.103454,4.762190,-1.091029,bonafide,LA_D_1364611
3,-11.137697,-3.057932,-5.911511,-7.392115,3.996975,13.637284,-0.424157,-16.094330,0.244491,22.745517,...,10.576336,-2.574009,-7.821240,0.479549,8.566047,9.951756,3.530564,-2.890661,bonafide,LA_D_3986002
4,-14.439103,-28.854320,0.629952,4.307006,10.547094,5.241080,7.606417,7.646509,7.594661,7.438350,...,9.231278,11.949671,10.503574,7.860767,6.397136,1.222892,1.805485,8.955902,spoof,LA_D_3411825
5,-11.984894,-2.999891,18.083846,-5.197033,-8.321549,-24.664801,-24.898103,-12.564918,-4.009618,-0.710744,...,-26.809098,-30.535644,-13.446531,-5.868994,-0.810060,-5.107424,-6.477157,-5.269687,spoof,LA_D_8173305
6,-11.745431,-5.605191,2.593292,-5.325038,5.581158,5.016391,2.122084,-3.730798,0.550565,9.172164,...,10.340688,4.697443,4.404067,8.119862,3.701606,1.727328,2.000289,2.063593,bonafide,LA_D_6027798


In [82]:
feature_len = 650

In [86]:
X = data.iloc[:, 0:feature_len].values
y = data[['650']].values

In [87]:
y

array([['spoof'],
       ['bonafide'],
       ['bonafide'],
       ['bonafide'],
       ['spoof'],
       ['spoof'],
       ['bonafide']], dtype=object)

In [88]:
svm = SVC(probability=True)	

In [89]:
svm.fit(X, y)

/Users/asimadnan/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVC(probability=True)

In [90]:
# SVM Test

In [ ]:
# CNN

In [136]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D,Conv1D
from keras.models import model_from_json

In [92]:
def extract_data(data_path,label_path,feature_type):
    data_set = pd.read_csv(data_path, delimiter = ",",index_col=0)
    labels = pd.read_csv(label_path, delimiter = " ", header=None)
    labels.columns = ['SPEAKER_ID','AUDIO_FILE_NAME','ENVIRONMENT_ID','ATTACK_ID','KEY']

    feature_len = 650

    if (feature_type == 'cqcc'):
    	feature_len = 3000

    
    data_set = data_set.rename({str(feature_len + 1): 'AUDIO_FILE_NAME'}, axis=1)
    data_set = pd.merge(data_set, labels, on='AUDIO_FILE_NAME')
    
    return data_set.iloc[:, 0:feature_len].values,data_set.iloc[:, feature_len+1:data_set.shape[1]]


In [94]:
feature = 'mfcc'
dataset = 'test'
###
train_data_mfcc_path = '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/output_data/test__mfcc.csv'
train_data_cqcc_path= '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/output_data/test__cqcc.csv'
dev_label_path = '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.dev.trl.txt'


raw_data,labels = extract_data(train_data_mfcc_path,dev_label_path,feature)

In [120]:
x_train, y_train = raw_data,labels[['KEY']].values
x_test, y_test = raw_data,labels[['KEY']].values

In [128]:
a = (y_train == 'bonafide')

flat_list = [item for sublist in a for item in sublist]

y_train = flat_list

a = (y_test == 'bonafide')

flat_list = [item for sublist in a for item in sublist]

y_test = flat_list

In [132]:
y_train = 1*np.array(y_train)
y_test = 1*np.array(y_test)

In [133]:
#x_train, y_train, x_test, y_test = create_train_test(path)

print("Size of Training Data:", np.shape(x_train))
print("Size of Training Labels:", np.shape(y_train))
print("Size of Test Data:", np.shape(x_test))
print("Size of Test Labels:", np.shape(y_test))

# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

# x_train = x_train.reshape(x_train.shape[0], imheight, imwidth, 1)
# x_test = x_test.reshape(x_test.shape[0], imheight, imwidth, 1)
input_shape = (1, 1, 650)
batch_size = 4
epochs = 1
num_classes = 2

Size of Training Data: (7, 650)
Size of Training Labels: (7,)
Size of Test Data: (7, 650)
Size of Test Labels: (7,)


In [177]:
model = Sequential()
model.add(Dense(128, input_dim=650, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
# model.add(Dense(8, activation='relu'))
# model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='softplus'))


model.compile(loss=keras.losses.categorical_crossentropy,  metrics=['accuracy'])
print(model.summary())

model.fit(x_train, y_train, batch_size=4, epochs=10, verbose=1, validation_data=(x_test, y_test))

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 128)               83328     
_________________________________________________________________
dense_56 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_57 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_58 (Dense)             (None, 1)                 65        
Total params: 108,161
Trainable params: 108,161
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
2/2 [==============================] - 1s 330ms/step - loss: 6.5281e-08 - accuracy: 0.5476 - val_loss: 6.8120e-08 - val_accuracy: 0.5714
Epoch 2/10
2/2 [==============================] - 0s 112ms/step - loss: 6.5281e-08 - accurac

In [160]:
model.save('nnn',save_format='h5')



In [142]:
reconstructed_model = keras.models.load_model("/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/output_data/mfcccnn_model")


In [146]:
raw_data,labels = extract_data(train_data_mfcc_path,dev_label_path,'mfcc')
x_test, y_test = raw_data,labels[['KEY']].values
a = (y_test == 'bonafide')
flat_list = [item for sublist in a for item in sublist]
y_test = flat_list
y_test = 1*np.array(y_test)

In [147]:
results = reconstructed_model.evaluate(x_test, y_test, batch_size=128)

2/2 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.4286


In [155]:
predictions = reconstructed_model.predict(x_test)
predictions

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)

In [178]:
test_prob = model.predict(x_test)

In [179]:
test_prob

array([[14.217662 ],
       [10.117344 ],
       [ 6.7160873],
       [13.3848915],
       [13.181759 ],
       [13.335044 ],
       [ 8.605457 ]], dtype=float32)

In [2]:
# reading in lfcc

In [3]:
lfcc = '/Users/asimadnan/Desktop/Mres/ASVspoof_2019_baseline_CM_v1/cqcc_mat.txt'


In [5]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [6]:
len  = file_len(lfcc)



In [7]:
len

2580

In [11]:
data = []
path = lfcc
files = [lfcc]
for f in files:
  with open (f, "r") as myfile:
    data.append(myfile.read())

df = pd.DataFrame(data)

In [5]:
lfcc = '/Users/asimadnan/Desktop/Mres/ASVspoof_2019_baseline_CM_v1/lfcc_bon_train.txt'

In [19]:
data = []
with open(lfcc) as input_file:
  for row in input_file:
    data.append(row.split(','))
   

In [20]:
data.shape

AttributeError: 'list' object has no attribute 'shape'